<a href="https://colab.research.google.com/github/ram130849/llm-finetuning/blob/main/zephyr_7b_alpha_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU transformers datasets trl peft accelerate bitsandbytes auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 35.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

In [3]:
MODEL_ID = "TheBloke/zephyr-7B-alpha-GPTQ"
DATASET_ID = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"
CONTEXT_FIELD= ""
INSTRUCTION_FIELD = "instruction"
TARGET_FIELD = "response"
BITS = 4
DISABLE_EXLLAMA = True
DEVICE_MAP = "auto"
USE_CACHE = False
LORA_R = 16
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
BIAS = "none"
TARGET_MODULES = ["q_proj", "v_proj"]
TASK_TYPE = "CAUSAL_LM"
OUTPUT_DIR = "zephyr-support-chatbot"
BATCH_SIZE = 8
GRAD_ACCUMULATION_STEPS = 1
OPTIMIZER = "paged_adamw_32bit"
LR = 2e-4
LR_SCHEDULER = "cosine"
LOGGING_STEPS = 50
SAVE_STRATEGY = "epoch"
NUM_TRAIN_EPOCHS = 3
MAX_STEPS = 250
FP16 = True
PUSH_TO_HUB = True
DATASET_TEXT_FIELD = "text"
MAX_SEQ_LENGTH = 1024
PACKING = False

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
data = load_dataset(DATASET_ID,split='train')
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 26872
})

In [7]:
df = data.to_pandas()
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [8]:
def process_data_sample(example):
    processed_example = "<|system|>\n You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.\n<|user|>\n" + example[INSTRUCTION_FIELD] + "\n<|assistant|>\n" + example[TARGET_FIELD]
    return processed_example

In [9]:
df[DATASET_TEXT_FIELD] = df[[INSTRUCTION_FIELD, TARGET_FIELD]].apply(lambda x: process_data_sample(x), axis=1)
df.head()
print(df.iloc[0]['text'])

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
question about cancelling order {{Order Number}}
<|assistant|>
I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.


In [10]:
processed_data = Dataset.from_pandas(df[[DATASET_TEXT_FIELD]])
processed_data

Dataset({
    features: ['text'],
    num_rows: 26872
})

In [8]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/zephyr-7B-alpha-GPTQ")
tokenizer.pad_token = tokenizer.eos_token

In [12]:
bnb_config = GPTQConfig(bits=4,
                        disable_exllama=True,
                        device_map="auto",
                        use_cache=False,
                        lora_r=16,
                        lora_alpha=16,
                        tokenizer=tokenizer
                                )
#
model = AutoModelForCausalLM.from_pretrained("TheBloke/zephyr-7B-alpha-GPTQ",
                                              quantization_config=bnb_config,
                                              device_map="auto",
                                              use_cache=False,
                                              )

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [13]:
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
print("\n====================================================================\n")
print("\t\t\tMODEL CONFIG UPDATED")
print("\n====================================================================\n")

peft_config = LoraConfig(
                            r=LORA_R,
                            lora_alpha=LORA_ALPHA,
                            lora_dropout=LORA_DROPOUT,
                            bias=BIAS,
                            task_type=TASK_TYPE,
                            target_modules=TARGET_MODULES
                        )

model = get_peft_model(model, peft_config)
print("\n====================================================================\n")
print("\t\t\tPREPARED MODEL FOR FINETUNING")
print(model)
print("\n====================================================================\n")



			MODEL CONFIG UPDATED




			PREPARED MODEL FOR FINETUNING
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (rotary_emb): MistralRotaryEmbedding()
              (k_proj): QuantLinear()
              (o_proj): QuantLinear()
              (q_proj): QuantLinear(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embeddi

In [14]:
training_arguments = TrainingArguments(
                                        output_dir=OUTPUT_DIR,
                                        per_device_train_batch_size=BATCH_SIZE,
                                        gradient_accumulation_steps=GRAD_ACCUMULATION_STEPS,
                                        optim=OPTIMIZER,
                                        learning_rate=LR,
                                        lr_scheduler_type=LR_SCHEDULER,
                                        save_strategy=SAVE_STRATEGY,
                                        logging_steps=LOGGING_STEPS,
                                        num_train_epochs=NUM_TRAIN_EPOCHS,
                                        max_steps=MAX_STEPS,
                                        fp16=FP16,
                                        push_to_hub=PUSH_TO_HUB)

In [15]:
device="cuda"
model.to(device)

trainer = SFTTrainer(
                        model=model,
                        train_dataset=processed_data,
                        peft_config=peft_config,
                        dataset_text_field=DATASET_TEXT_FIELD,
                        args=training_arguments,
                        tokenizer=tokenizer,
                        packing=PACKING,
                        max_seq_length=MAX_SEQ_LENGTH
                    )
trainer.train()

Map:   0%|          | 0/26872 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:214: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the diff

Step,Training Loss
50,0.947700
100,0.682500


Step,Training Loss
50,0.947700
100,0.682500
150,0.637200
200,0.604300
250,0.581200


TrainOutput(global_step=250, training_loss=0.69058642578125, metrics={'train_runtime': 2388.8131, 'train_samples_per_second': 0.837, 'train_steps_per_second': 0.105, 'total_flos': 534655918080000.0, 'train_loss': 0.69058642578125, 'epoch': 0.07})

In [11]:
def process_data_sample(example):

    processed_example = "<|system|>\n You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.\n<|user|>\n" + example["instruction"] + "\n<|assistant|>\n"

    return processed_example

# tokenizer = AutoTokenizer.from_pretrained("/content/zephyr-support-chatbot")

inp_str = process_data_sample(
    {
        "instruction": "i have a question about cancelling order {{Order Number}}",
    }
)


tokenizer = AutoTokenizer.from_pretrained("/ramrama/zephyr-support-chatbot")


inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")


model = AutoPeftModelForCausalLM.from_pretrained(
    "ramrama/zephyr-support-chatbot",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

In [12]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
i have a question about cancelling order {{Order Number}}
<|assistant|>
I've understood that you have a question about canceling order {{Order Number}}. I'm here to assist you with that. To cancel your order, you can reach out to our customer support team. They will guide you through the process and ensure that your order is canceled smoothly. If you have any specific concerns or need further assistance, please don't hesitate to let me know. I'm here to help you every step of the way. Thank you for reaching out to us. How else can I assist you today?
14.921103477478027


In [16]:
inp_str = process_data_sample(
    {
        "instruction": "i have a question about the delay in order 10899",
    }
)

inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

In [17]:
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
i have a question about the delay in order 10899
<|assistant|>
I'm aware that you have a question regarding the delay in order {{Order Number}}. I apologize for any inconvenience this may have caused. To provide you with the most accurate information, could you please provide me with more details about the specific issue you are facing? This will help me better understand the situation and assist you accordingly. Your satisfaction is our top priority, and we are committed to resolving any concerns you may have. Thank you for bringing this to our attention, and I appreciate your patience as we work towards a solution.
